In [5]:
import pandas as pd
import math
import os

def perform_stratified_sampling(sample_fraction, input_file, output_file, min_per_group=1, stratify_col='tier', random_state=42):
    """
    계층적 샘플링 수행 (기본: 티어별 샘플링).
    - sample_fraction: 각 그룹에서 비율로 뽑을 비율
    - min_per_group: 그룹이 작을 때 최소 보장 샘플 수
    - stratify_col: 계층화에 사용할 컬럼 이름
    """
    print('📂 데이터 로딩 중...')
    df = pd.read_csv(input_file)

    # NaN 처리를 안전하게 함
    if stratify_col not in df.columns:
        raise ValueError(f"'{stratify_col}' 컬럼이 입력 파일에 없습니다.")
    df[stratify_col] = df[stratify_col].fillna('Unknown')

    # 그룹별로 샘플 수 계산
    def _sample_group(g):
        grp_size = len(g)
        n = max(0, int(round(grp_size * sample_fraction)))
        if n < min_per_group and grp_size > 0:
            n = min(min_per_group, grp_size)
        if n <= 0:
            return g.iloc[0:0]
        return g.sample(n=n, random_state=random_state)

    sampled_df = df.groupby(stratify_col, group_keys=False).apply(_sample_group).reset_index(drop=True)

    # 결과 출력 및 저장
    print('\n✅ 샘플링 완료!')
    print(f"전체 데이터 수: {len(df):,} 명")
    print(f"샘플링 데이터 수: {len(sampled_df):,} 명")
    print(f"샘플 내 {stratify_col} 비중:\n{sampled_df[stratify_col].value_counts(normalize=True)}")

    sampled_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"💾 샘플 데이터가 {output_file}에 저장되었습니다.")

In [6]:
name = 'kr_puuids_high_tiers'
sampling = 0.1
perform_stratified_sampling(
    sample_fraction = sampling,
    input_file=f'{name}.csv', 
    output_file=f'{sampling*100}%_{name}.csv'
)

📂 데이터 로딩 중...

✅ 샘플링 완료!
전체 데이터 수: 11,000 명
샘플링 데이터 수: 1,100 명
샘플 내 tier 비중:
tier
MASTER         0.909091
GRANDMASTER    0.063636
CHALLENGER     0.027273
Name: proportion, dtype: float64
💾 샘플 데이터가 10.0%_kr_puuids_high_tiers.csv에 저장되었습니다.


C:\Users\andor\AppData\Local\Temp\ipykernel_23644\1881658727.py:30: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby(stratify_col, group_keys=False).apply(_sample_group).reset_index(drop=True)
